In [1]:
import os
import sys
import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import nibabel as nib
from scipy import ndimage

In [2]:
print("CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())

CUDA available: True
Number of GPUs: 1


In [3]:

def read_nifti_file(filepath):
    """Read and load volume"""
    scan = nib.load(filepath)
    scan = scan.get_fdata()
    return scan

def normalize(volume):
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume

def resize_volume(img):
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    img = ndimage.rotate(img, 90, reshape=False)
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img

def process_scan(path):
    volume = read_nifti_file(path)
    volume = normalize(volume)
    volume = resize_volume(volume)
    return volume

In [4]:
normal_scan_paths = [
    os.path.join(os.getcwd(), "MosMedData/CT-0", x)
    for x in os.listdir("MosMedData/CT-0")if x[-1] == 'z'
]
abnormal_scan_paths = [
    os.path.join(os.getcwd(), "MosMedData/CT-23", x)
    for x in os.listdir("MosMedData/CT-23")if x[-1] == 'z'
]
print("CT scans with normal lung tissue: " + str(len(normal_scan_paths)))
print("CT scans with abnormal lung tissue: " + str(len(abnormal_scan_paths)))




CT scans with normal lung tissue: 254
CT scans with abnormal lung tissue: 856


In [5]:
# Read and process the scans.
# Each scan is resized across height, width, and depth and rescaled.
normal_scans = np.array([process_scan(path) for path in normal_scan_paths])
# assign 1, for the normal ones assign 0.
normal_labels = np.array([0 for _ in range(len(normal_scans))])
norm_split = round(len(normal_scans) * .9 )

In [6]:
abnormal_scans = np.array([process_scan(path) for path in abnormal_scan_paths])
abnormal_labels = np.array([1 for _ in range(len(abnormal_scans))])
abnorm_split = round(len(abnormal_scans) * .9 )

In [7]:
print("Normal scans in x train",normal_scans[:norm_split].shape)
print("Normal scans in X val ",normal_scans[norm_split:].shape)

print("Abormal scans in x train",abnormal_scans[:abnorm_split].shape)
print("Abormal scans in X val ",abnormal_scans[abnorm_split:].shape)


Normal scans in x train (229, 128, 128, 64)
Normal scans in X val  (25, 128, 128, 64)
Abormal scans in x train (770, 128, 128, 64)
Abormal scans in X val  (86, 128, 128, 64)


In [8]:
# Split data in the ratio 70-30 for training and validation.
x_train = np.concatenate((abnormal_scans[:abnorm_split], normal_scans[:norm_split]), axis=0)
y_train = np.concatenate((abnormal_labels[:abnorm_split], normal_labels[:norm_split]), axis=0)
x_val = np.concatenate((abnormal_scans[abnorm_split:], normal_scans[norm_split:]), axis=0)
y_val = np.concatenate((abnormal_labels[abnorm_split:], normal_labels[norm_split:]), axis=0)

In [9]:
print("x train shape", x_train.shape)
print("y train shape", y_train.shape)
print("x val shape", x_val.shape)
print("y val shape", y_val.shape)

x train shape (999, 128, 128, 64)
y train shape (999,)
x val shape (111, 128, 128, 64)
y val shape (111,)


In [10]:
print("Shape of abnormal scans:", abnormal_scans.shape)
print("Shape of normal scans:", normal_scans.shape)


Shape of abnormal scans: (856, 128, 128, 64)
Shape of normal scans: (254, 128, 128, 64)


In [11]:
class CTScansDataset(Dataset):
    def __init__(self, x, y, transform=None):
        self.x = x
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = self.x[idx]
        label = self.y[idx]

        if self.transform:
            image = self.transform(image)
        image = torch.from_numpy(image)
        image = image.unsqueeze(0)

        return (image, label)

# class RandomRotation3D:
#     def __init__(self, angles):
#         self.angles = angles

#     def __call__(self, image):
#         angle = np.random.choice(self.angles)
#         image = ndimage.rotate(image, angle, reshape=False)
#         image[image < 0] = 0
#         image[image > 1] = 1
#         return image


class NumpyToTensor:
    def __call__(self, image):
        return torch.from_numpy(image)

In [12]:
# train_transform = transforms.Compose([
# RandomRotation3D(angles=[-20, -10, -5, 5, 10, 20]),
# NumpyToTensor(),
# ])

# train_transform = transforms.Compose([
# NumpyToTensor(),
# ])
# val_transform = transforms.Compose([
# NumpyToTensor(),
# ])


train_transform = None
val_transform = None 

train_dataset = CTScansDataset(x_train, y_train, transform=train_transform)
val_dataset = CTScansDataset(x_val, y_val, transform=val_transform)

batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [13]:
from resnet import ResNet, BasicBlock


class ModifiedResNet(ResNet):

    def __init__(self, *args, **kwargs):
        super(ModifiedResNet, self).__init__(*args, **kwargs)
        
        self.avgpool = nn.AdaptiveAvgPool3d(1)
        self.fc = nn.Linear(512 * kwargs["block"].expansion, 1)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.dropout(x) 
        x = self.layer2(x)
        x = self.dropout(x) 
        x = self.layer3(x)
        x = self.dropout(x) 
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        x = self.dropout(x) 
        x = torch.sigmoid(x)
        
        return x


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
learning_rate = 0.0001
# Create the model instance

model = ModifiedResNet(block=BasicBlock, layers=[3, 4, 6, 3], sample_input_W=128, sample_input_H=128, sample_input_D=64, num_seg_classes=1).to(device)


# Load the pre-trained model's state_dict
pretrained_state_dict = torch.load("pretrain/resnet_34_23dataset.pth")


# Check if the state_dict is saved under the "state_dict" key and extract it
if "state_dict" in pretrained_state_dict:
    pretrained_state_dict = pretrained_state_dict["state_dict"]


# Remove 'module.' prefix from keys if it exists
corrected_state_dict = {k.replace("module.", ""): v for k, v in pretrained_state_dict.items()}


# Find the keys present in both the pre-trained model and your ModifiedResNet model
common_keys = set(corrected_state_dict.keys()) & set(model.state_dict().keys())


# Create a new state_dict with only the common keys
filtered_state_dict = {k: corrected_state_dict[k] for k in common_keys}


# Load the filtered state_dict into your model, allowing for partial weights with strict=False
model.load_state_dict(filtered_state_dict, strict=False)


# # Wrap the model with DataParallel
if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model)


criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.0001)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.96)



cuda


In [15]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_confusion_matrix(conf_matrix, class_names):
    fig, ax = plt.subplots(figsize=(6, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", ax=ax, cbar=False)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    ax.set_xticklabels(class_names)
    ax.set_yticklabels(class_names)
    plt.close(fig)
    return fig

In [17]:
from sklearn.metrics import confusion_matrix, f1_score, recall_score
import tqdm
import ipdb
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

# ... (all previous code remains the same)
num_epochs = 100
best_val_acc = 0.0
best_epoch = 0

for epoch in range(num_epochs):
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
            data_loader = train_loader
            dataset = train_dataset
        else:
            model.eval()
            data_loader = validation_loader
            dataset = val_dataset

        running_loss = 0.0
        running_corrects = 0
        all_preds = []
        all_labels = []

        for inputs, labels in tqdm.tqdm(data_loader, desc=f"{phase.capitalize()} Epoch {epoch}"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                outputs = outputs.view(-1)
                loss = criterion(outputs, labels.float())
                
                preds = (outputs > 0.5).float()

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            # ipdb.set_trace(context=5)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        epoch_loss = running_loss / len(dataset)
        epoch_acc = running_corrects.double() / len(dataset)

        if phase == 'val':
            # Calculate confusion matrix, F1 score, and recall
            conf_matrix = confusion_matrix(all_labels, all_preds)
            f1 = f1_score(all_labels, all_preds, average='weighted')
            recall = recall_score(all_labels, all_preds, average='weighted')

            # Print confusion matrix, F1 score, and recall
            print(f"Confusion matrix for epoch {epoch}:\n{conf_matrix}")
            print(f'Epoch {epoch}/{num_epochs - 1}, {phase.capitalize()} F1 Score: {f1:.4f}, {phase.capitalize()} Recall: {recall:.4f}')

            # Log confusion matrix to TensorBoard
            writer.add_figure("Confusion Matrix/val", plot_confusion_matrix(conf_matrix, ["Normal", "Abnormal"]), epoch)

        if phase == 'train':
            scheduler.step()
            writer.add_scalar('Loss/train', epoch_loss, epoch)
            writer.add_scalar('Accuracy/train', epoch_acc, epoch)
        else:
            writer.add_scalar('Loss/val', epoch_loss, epoch)
            writer.add_scalar('Accuracy/val', epoch_acc, epoch)

        print(f'Epoch {epoch}/{num_epochs - 1}, {phase.capitalize()} Loss: {epoch_loss:.4f}, {phase.capitalize()} Accuracy: {epoch_acc:.4f}')

        if phase == 'val' and epoch_acc > best_val_acc:
            best_val_acc = epoch_acc
            best_epoch = epoch
            best_model_wts = model.state_dict()

torch.save(best_model_wts, '3d_image_classification.pth')
print(f"Training complete. Best validation accuracy: {best_val_acc:.4f} at epoch {best_epoch}")


Train Epoch 0: 100%|██████████| 250/250 [01:29<00:00,  2.80it/s]


Epoch 0/99, Train Loss: 0.5572, Train Accuracy: 0.7187


Val Epoch 0: 100%|██████████| 28/28 [00:04<00:00,  6.57it/s]


Confusion matrix for epoch 0:
[[ 0 25]
 [ 0 86]]
Epoch 0/99, Val F1 Score: 0.6765, Val Recall: 0.7748
Epoch 0/99, Val Loss: 0.5300, Val Accuracy: 0.7748


Train Epoch 1: 100%|██████████| 250/250 [01:36<00:00,  2.60it/s]


...

Confusion matrix for epoch 98:
[[ 7 18]
 [ 4 82]]
Epoch 98/99, Val F1 Score: 0.7707, Val Recall: 0.8018
Epoch 98/99, Val Loss: 0.8064, Val Accuracy: 0.8018


Train Epoch 99: 100%|██████████| 250/250 [01:38<00:00,  2.55it/s]


Epoch 99/99, Train Loss: 0.0926, Train Accuracy: 0.9219


Val Epoch 99: 100%|██████████| 28/28 [00:03<00:00,  7.54it/s]


Confusion matrix for epoch 99:
[[ 9 16]
 [ 6 80]]
Epoch 99/99, Val F1 Score: 0.7825, Val Recall: 0.8018
Epoch 99/99, Val Loss: 0.7417, Val Accuracy: 0.8018
Training complete. Best validation accuracy: 0.8378 at epoch 22
